In [2]:
import numpy as np
import os
import tensorflow as tf

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

###### Do not modify here ###### 
y_train1 = y_train1[:, np.newaxis]
y_valid1 = y_valid1[:, np.newaxis]
y_test1 = y_test1[:, np.newaxis]
_, dim = X_train1.shape
print(dim)

# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, dim])
ys = tf.placeholder(tf.float32, [None, 1])

W = tf.Variable(tf.zeros([784, 1]))
b = tf.Variable(tf.zeros([1]))
y = tf.nn.softmax(tf.matmul(xs, W) + b)

cross_entropy = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits = y, labels = ys))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

sess = tf.InteractiveSession()

tf.global_variables_initializer().run()
for i in range(1000):
    if(i%100==0):
        print("Steps:",i)
    #batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={xs: X_train1, ys: y_train1})

print(sess.run(y, feed_dict={xs: X_train1}))
correct_prediction = tf.equal(y, ys)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={xs: X_train1, ys: y_train1}))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
784
Steps: 0
Steps: 100
Steps: 200
Steps: 300
Steps: 400
Steps: 500
Steps: 600
Steps: 700
Steps: 800
Steps: 900
[[ 1.]
 [ 1.]
 [ 1.]
 ..., 
 [ 1.]
 [ 1.]
 [ 1.]]
0.220379
